In [1]:
import os
import pathlib
import shutil
import json
import polars as pl
from pprint import pprint
from tqdm import tqdm

pl.Config.load_from_file("pl_config.json")
pl.set_random_seed(999)

## 다운로드

다운받을 데이터셋 정보 조회

In [2]:
!aihubshell -mode l -datasetkey 71263

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터
        ├─02.저작도구
        │  ├─저작도구 소스코드.zip | 1 MB | 61182
        │  └─저작도구 설명서.zip | 687 MB | 61183
        └─01.데이터
            ├─1.Training
            │  ├─라벨링데이터
            │  │  ├─TL1.zip | 44 MB | 61236
            │  │  ├─TL2.zip | 32 MB | 61237
            │  │  ├─TL3.zip | 44 MB | 61238
            │  │  └─TL4.zip | 37 MB | 61239
            │  └─원천데이터
            │      ├─TS1.zip | 44 MB | 61240
            │      ├─TS2.zip | 32 MB | 61241
            │      ├─TS3.zip | 44 MB | 61242
            │      └─TS4.zip | 37 MB | 61243
            └─2.Validation
                ├─라벨링데이터
                │  └─VL1.zip | 20 MB | 61244
                └─원천데이터
               

데이터 개요는 이 [페이지](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71263)에서 찾아볼 수 있다. 라벨링데이터를 다운받아 분석을 진행한다.

In [3]:
!aihubshell -mode d -datasetkey 71263 -filekey 61236,61237,61238,61239

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  158M    0  158M    0     0  4537k      0 --:--:--  0:00:35 --:--:-- 6153k     0  3735k      0 --:--:--  0:00:14 --:--:-- 3497k:--  0:00:20 --:--:-- 4687k
Request successful with HTTP status 200.
Download successful.
x 157.방송_콘텐츠_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1.Training/라벨링데이터/TL4.zip.part0
x 157.방송_콘텐츠_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1.Training/라벨링데이터/TL3.zip.part0
x 157.방송_콘텐츠_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1.Training/라벨링데이터/TL2.zip.part0
x 157.방송_콘텐츠_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1.Training/라벨링데이터/TL1.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging TL1.zip in ./157.방송_콘텐츠_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1.Training/라벨링데이터
Merging TL2.zip in ./157.방송_콘텐츠_한-중,_한-일_번역_병렬_말뭉치_데이터/01.데이터/1.Training/라벨링데이터
M

다운받은 데이터 디렉토리 및 결과 디렉토리 정의

In [4]:
archive_dir = [path for path in pathlib.Path(os.getcwd()).glob("157*") if path.suffix != ".ipynb"][0]
result_dir = pathlib.Path(f"{os.getcwd()}/aihub-datasets")
result_dir.mkdir(parents=True, exist_ok=True)

## 압축 해제 및 EDA

데이터 압축파일 경로 정의

In [5]:
archive_paths = []
for walk in os.walk(archive_dir):
    current_path, folders, files = walk
    for file_name in files:
        if file_name.endswith(".zip") and current_path.endswith("라벨링데이터"):
            archive_paths.append(f"{current_path}/{file_name}")

압축 해제

In [6]:
for archive_path in archive_paths:
    data_dir = pathlib.Path(archive_path.strip(".zip"))
    shutil.unpack_archive(
        filename=archive_path,
        extract_dir=data_dir,
        format="zip",
    )

각 데이터는 json 형태로 저장되어 있다.

In [7]:
data_files = list(data_dir.parent.glob("*/*.json"))
print(f"Number of files in dataset : {len(data_files):,}")

Number of files in dataset : 4


데이터 속성값들. 카테고리별로 한국어-일본어 문장 페어가 다른 특징들과 함께 포함되어 있는 것을 확인할 수 있다. 

* 이 데이터셋은 한-중 번역문도 포함하고 있기 때문에, 예를 들어 한-일 번역문만 사용하고자 한다면  (source_language, target_language)가 (ko, jp) 또는 (jp, ko)인 데이터만 사용해야 한다.
* 알수없는 이유로 원문과 번역문 시작에 `>`가 포함되어 있는 경우가 많은데, 제거하는 전처리가 필요해 보인다.
* ner 태그가 함께 붙어있는 경우도 있다.

In [8]:
with open(data_files[0], "r") as file:
    documents = json.load(file)
pprint(documents["data"][:3])

[{'data_set': '방송콘텐츠',
  'domain': 'TV방송',
  'file_name': '관찰예능_22_8000.xlsx',
  'included_unknown_words': False,
  'jp': '>ジャンフンさんみたいなご主人となかなか会えないですよ。',
  'ko': '>우리 장훈 씨 같은 남편을 어디서 만나요.',
  'ko_original': '>우리 장훈 씨 같은 남편을 어디서 만나요.',
  'license': 'open',
  'mt': '>ジャンフンさんみたいなご主人となかなか会えないですよ。',
  'ner': {'tags': [{'position': '[4, 6]', 'tag': 'PERSON', 'value': '장훈'}],
          'text': '>우리 <PERSON>장훈</PERSON> 씨 같은 남편을 어디서 만나요.'},
  'sn': 'KVSMUS171V5501',
  'source': 'SBS',
  'source_language': 'ko',
  'style': '구어체',
  'subdomain': '관찰예능',
  'target_language': 'jp',
  'word_count_jp': 7,
  'word_count_ko': 7,
  'word_ratio': 1},
 {'data_set': '방송콘텐츠',
  'domain': 'TV방송',
  'file_name': '관찰예능_31_8000.xlsx',
  'included_unknown_words': False,
  'jp': '>いや、いや。',
  'ko': '>아니, 아니.',
  'ko_original': '>아니, 아니.',
  'license': 'open',
  'mt': 'ううん、ううん。',
  'ner': None,
  'sn': 'KVSMUS208EE7164',
  'source': 'SBS',
  'source_language': 'ko',
  'style': '구어체',
  'subdomain': '관찰예능',
  'targe

전체 로드(첫 글자에 있는 `>`는 제거하는 전처리를 같이 적용했다)

In [9]:
corpus = []
for data_file in tqdm(data_files):
    with open(data_file, "r") as file:
        documents = json.load(file)["data"]
    for document in documents:
        source = document.pop("source_language")
        target = document.pop("target_language")
        if (source == "ko" and target == "jp") or (source == "jp" and target == "ko"):
            korean = document.pop("ko")
            korean = korean[1:] if korean[0] in ">〉＞" else korean
            japanese = document.pop("jp")
            japanese = japanese[1:] if japanese[0] in ">〉＞" else japanese
            data = {
                "korean": korean,
                "japanese": japanese,
                "category_1st": document.get("domain", None),
                "category_2nd": document.get("subdomain", None),
            }
            corpus.append(data)

100%|█████████████████████████████████████████████| 4/4 [00:09<00:00,  2.28s/it]


EDA를 위해 데이터프레임으로 변환

In [10]:
eda_dataset = pl.DataFrame(corpus).sample(fraction=1, shuffle=True)
print(f"Number of documents in the corpus : {eda_dataset.height:,}")

Number of documents in the corpus : 960,000


### 카테고리별 문서 수 분포

카테고리별 문서 수 분포는 아래와 같았다. 사용하려는 필요에 따라 이 중 특정 카테고리에서 발생한 문서들은 제외해도 무방할 듯 하다.

In [11]:
(
    eda_dataset
    .group_by(["category_1st", "category_2nd"])
    .len("num_documents")
    .sort(by=["category_1st", "category_2nd"])
)

category_1st,category_2nd,num_documents
str,str,u32
"""TV방송""","""관찰예능""","279,846"
"""TV방송""","""교양""","159,913"
"""TV방송""","""리얼버라이어티예능""","160,241"
"""라디오방송""","""경제""","16,051"
"""라디오방송""","""문화""","144,077"
"""라디오방송""","""사회""","55,873"
"""라디오방송""","""일상""","143,999"


### 문서 토큰 수 분포 확인

한국어에서는 정확히 적용되지 않지만, 편의를 위해 어절의 수(`공백 수 + 1`)를 토큰 수라고 정의한다. 한국어 토큰이 약 530만개 가량 포함된 corpus임을 확인했다.

In [12]:
eda_dataset = eda_dataset.with_columns(pl.col("korean").str.replace(r"\s+", " "))
num_tokens = (
    eda_dataset
    .with_columns((pl.col("korean").str.count_matches(" ") + 1).alias("num_tokens"))
    .select("num_tokens")
)
print(f"Number of tokens in the Korean corpus : {num_tokens['num_tokens'].sum():,}")

Number of tokens in the Korean corpus : 5,228,298


문서당 토큰 수 분포

In [13]:
num_tokens.select(
    pl.col("num_tokens").min().alias("min"),
    pl.col("num_tokens").quantile(0.05).cast(pl.Int32).alias("q05"),
    pl.col("num_tokens").quantile(0.25).cast(pl.Int32).alias("Q1"),
    pl.col("num_tokens").quantile(0.50).cast(pl.Int32).alias("median"),
    pl.col("num_tokens").quantile(0.75).cast(pl.Int32).alias("Q3"),
    pl.col("num_tokens").quantile(0.95).cast(pl.Int32).alias("q95"),
    pl.col("num_tokens").max().alias("max"),
)

min,q05,Q1,median,Q3,q95,max
u32,i32,i32,i32,i32,i32,u32
1,1,2,4,7,16,136


토큰 한개짜리 문장들도 outlier라기보단 구어체에서 흔히 나타날 수 있는 문장임을 확인

In [14]:
(
    eda_dataset
    .with_columns((pl.col("korean").str.count_matches(" ") + 1).alias("num_tokens")).filter(pl.col("num_tokens") == 1)
    .sample(10)
    .select(["korean", "japanese"])
)

korean,japanese
str,str
"""진짜요?""","""本当ですか？"""
"""대박이다.""","""すごい。"""
"""뭐야?""","""何？"""
"""괜찮냐?""","""大丈夫？"""
"""너!""","""君！"""
"""이야!""","""すごい！"""
"""괜찮아?""","""大丈夫?"""
"""소울아~""","""ソウルちゃん~"""
"""그래.""","""そうだね。"""


### 마스킹 토큰 존재 여부 확인

다른 AI 허브 데이터같이 비식별화가 필요한 텍스트에 마스크가 적용되어 있는지 확인했다. 없다.

In [15]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([가-힣]+\))").alias("mask_pattern"))
    .filter(pl.col("mask_pattern").is_not_null())
    .group_by("mask_pattern")
    .len("pattern_count")
    .sort(by="pattern_count", descending=True)
    .head(10)
)

mask_pattern,pattern_count
str,u32
"""(목)""",121
"""(토)""",77
"""(웃음)""",66
"""(금)""",43
"""(월)""",19
"""(일)""",17
"""(화)""",13
"""(수)""",12
"""(명사)""",7


### 한국어에 일본어 텍스트가 포함된 경우

하지만 한국어 안에 일본어가 포함된 데이터가 일부 있었다.

In [16]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([一-龯ぁ-んァ-ン]+\))").alias("jp_pattern"))
    .filter(pl.col("jp_pattern").is_not_null())
    .group_by("jp_pattern")
    .len("pattern_count")
    .sort(by="pattern_count", descending=True)
    .head(10)
)

jp_pattern,pattern_count
str,u32
"""(五友歌)""",3
"""(子)""",2
"""(ミニョンヌ)""",2
"""(空)""",2
"""(春分)""",2
"""(川流)""",2
"""(藥水)""",1
"""(どうぞ)""",1
"""(頭)""",1


이런 패턴은 라디오방송 카테고리에서만 나타나는데, 이 카테고리는 일 -> 한 번역 텍스트에만 포함되어 있다. 이런 패턴이 주로 일 -> 한 번역 과정에서 나타나는 패턴임을 알 수 있다.

In [20]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([一-龯ぁ-んァ-ン]+\))").alias("jp_pattern"))
    .filter(pl.col("jp_pattern").is_not_null())
    .group_by("category_1st").len()
)

category_1st,len
str,u32
"""라디오방송""",44


한국어에는 한국어만 포함시키기 위해 이런 패턴은 전부 제거하는 전처리를 적용했다.

In [21]:
eda_dataset = eda_dataset.with_columns(pl.col("korean").str.replace(r"(\([一-龯ぁ-んァ-ン]+\))", ""))

## 데이터 저장

EDA를 마친 corpus를 앞서 정의한 `result_dir`에 같은 다운받은 데이터셋과 동일한 이름으로 저장

In [23]:
result_path = result_dir.joinpath(f"{archive_dir.name}.parquet")
eda_dataset.write_parquet(result_path)

로컬 디렉토리 정리

In [24]:
os.system(f"rm -rf {archive_dir}")

0

저장된 데이터셋에는 한국어 문장(`korean`), 이에 대응하는 일본어 문장(`japanese`)과 함께 해당 문장의 카테고리가 저장되어 있다.

In [25]:
eda_dataset.head(10)

korean,japanese,category_1st,category_2nd
str,str,str,str
"""그렇군요.""","""なるほどね、""","""라디오방송""","""문화"""
"""신경림씨는 1935년 충북의 농촌에서 태어났습니다.""","""申庚林さんは、1935年、忠清北道の農村に生まれました。""","""라디오방송""","""문화"""
"""무드살롱 연주곡으로 '한강 블루스'입니다.""","""ムードサロンの演奏曲で「漢江ブルース」です。""","""라디오방송""","""일상"""
"""아줌마로서는요.""","""アジュンマとしてはね。""","""라디오방송""","""경제"""
"""자기 남편은 나라를 위해 당당하게 목숨을 바쳤다고 하더랍니다.""","""自分の夫は国のために堂々と命を投げ出したと話していたそうです。""","""라디오방송""","""사회"""
"""나 지금 약간 저기 내 얼굴에 피골이 상접해진 게 느껴져, 지금.""","""俺今お腹と背中がくっつきそうだよ。""","""TV방송""","""교양"""
"""살짝 가져가면.""","""そっと持っていったら。""","""라디오방송""","""일상"""
"""피아노 학원, 태권도 학원 어?""","""ピアノ教室、テコンドー道場。""","""TV방송""","""관찰예능"""
"""한탄강이래.""","""漢灘江だって。""","""TV방송""","""리얼버라이어티예능"""
